In [1]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

import time

pd.set_option('display.max_columns', 50)

In [2]:
years = list(range(2024,2021, -1))
years

[2024, 2023, 2022]

In [3]:
connect_timeout = 6
read_timeout = 100

def stats(link, year):
     
    data = requests.get(link, timeout=(connect_timeout, read_timeout))
    soup = BeautifulSoup(data.text)
    
    stats_table = pd.read_html(data.text, match="Regular season Table")[0]
    
    standard_stats = pd.read_html(data.text, match = "Squad Standard Stats")[0]
    standard_stats.columns = standard_stats.columns.droplevel()
    
    goalkeeping_stats = pd.read_html(data.text, match="Squad Goalkeeping")[0]
    goalkeeping_stats.columns = goalkeeping_stats.columns.droplevel()
    
    shooting_stats = pd.read_html(data.text, match="Squad Shooting")[0]
    shooting_stats.columns = shooting_stats.columns.droplevel()
    
    stats_standard = stats_table.merge(standard_stats[["Squad", "Poss", "Gls", "Ast"]], on="Squad", how="left")
    stats_standard_gk = stats_standard.merge(goalkeeping_stats[["Squad", "CS%"]], on="Squad", how="left")
    all_stats = stats_standard_gk.merge(shooting_stats[["Squad", "SoT/90", "G/SoT", "SoT%"]], on="Squad", how="left")
        
    all_stats["Season"] = year
    
    return all_stats
    

In [4]:
overall_stats = []

In [5]:
def concate():
    
    all_stats_2024 = stats("https://fbref.com/en/comps/9/Premier-League-Stats", 2024)
    all_stats_2024 = all_stats_2024.drop(["Last 5"], axis=1)
    all_stats_2023 = stats("https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats", 2023)
    all_stats_2022 = stats("https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats", 2022)
    all_stats_2021 = stats("https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats", 2021)
    all_stats_2020 = stats("https://fbref.com/en/comps/9/2019-2020/2019-2020-Premier-League-Stats", 2020)
    all_stats_2019 = stats("https://fbref.com/en/comps/9/2018-2019/2018-2019-Premier-League-Stats", 2019)
    
    overall_stats.append(all_stats_2024)
    overall_stats.append(all_stats_2023)
    overall_stats.append(all_stats_2022)
    overall_stats.append(all_stats_2021)
    overall_stats.append(all_stats_2020)
    overall_stats.append(all_stats_2019)
    
    overall_stats_df = pd.concat(overall_stats)
    return overall_stats_df
    

In [6]:
overall_stats_df = concate()

In [7]:
overall_stats_df.head()

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes,Poss,Gls,Gls,Ast,Ast,CS%,SoT/90,G/SoT,SoT%,Season
0,1,Arsenal,31,22,5,4,75,24,51,71,2.29,61.3,21.3,40.0,1.29,60216,Bukayo Saka - 14,David Raya,NaN,59.8,71,2.29,50,1.61,45.2,5.32,0.38,32.5,2024
1,2,Liverpool,31,21,8,2,72,30,42,71,2.29,70.7,36.2,34.5,1.11,55211,Mohamed Salah - 17,Alisson,NaN,60.7,68,2.19,52,1.68,29.0,6.58,0.30,32.9,2024
2,3,Manchester City,31,21,7,3,71,31,40,70,2.26,62.9,30.1,32.8,1.06,52971,Erling Haaland - 19,Ederson,NaN,65.6,70,2.26,51,1.65,29.0,6.68,0.32,37.4,2024
3,4,Tottenham,31,18,6,7,65,45,20,60,1.94,54.6,50.0,4.6,0.15,57706,Son Heung-min - 15,Guglielmo Vicario,NaN,61.2,60,1.94,51,1.65,19.4,5.55,0.34,36.0,2024
4,5,Aston Villa,32,18,6,8,66,49,17,60,1.88,54.9,48.2,6.7,0.21,39318,Ollie Watkins - 18,Emiliano Martínez,NaN,54.7,63,1.97,46,1.44,21.9,5.09,0.36,36.5,2024


In [8]:
overall_stats_df.shape

(120, 29)

In [9]:
overall_stats_df.to_csv("../datasets/overall_stats2.csv", index=False)